<a href="https://colab.research.google.com/github/sznajder/CSC2026/blob/main/JetTagging_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a Jet Tagging **CNN**

---
In this notebook, we perform a Jet identification task using a CNN multiclass classifier.
The problem consists in identifying a given jet as a quark, a gluon, a W, a Z, or a top,
based on a jet image, i.e., a 2D histogram of the transverse momentum ($p_T$) deposited in each of 100x100
bins of a square window of the ($\eta$, $\phi$) plane, centered along the jet axis.

For details on the physics problem, see https://arxiv.org/pdf/1804.06913.pdf

Based on https://github.com/jngadiub/ML_course_Pavia_23

Modified by: Andre Sznajder

---

In [1]:
import os
import h5py
import glob, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Preparation of the training and validation samples

---
In order to import the dataset, we now
- clone the dataset repository (to import the data in Colab)
- load the h5 files in the data/ repository
- extract the data we need: a target and jetImage

To type shell commands, we start the command line with !

**nb, if you are running locally and you have already downloaded the datasets you can skip the cell below and, if needed, change the paths later to point to the folder with your previous download of the datasets.**

In [2]:
if not os.path.exists("Data-MLtutorial/JetDataset"):
    print("Dataset not found locally. Downloading and extracting...")
    os.system("curl -L https://cernbox.cern.ch/s/Hixs6KpgPFH8MN9/download -o Data-MLtutorial.tar.gz")
    os.system("tar -zxvf Data-MLtutorial.tar.gz")
    os.remove("Data-MLtutorial.tar.gz")
else:
    print("Dataset already exists. Skipping download.")

os.system("ls -al Data-MLtutorial/JetDataset/")

Dataset already exists. Skipping download.
total 748016
drwxr-xr-x  11 sznajder  staff       352 14 Aug 08:34 .
drwxr-xr-x   4 sznajder  staff       128  2 May  2022 ..
-rw-r--r--@  1 sznajder  staff      6148 14 Aug 08:32 .DS_Store
-rw-r--r--   1 sznajder  staff  47813131  2 May  2022 jetImage_7_100p_0_10000.h5
-rw-r--r--   1 sznajder  staff  48014611  2 May  2022 jetImage_7_100p_10000_20000.h5
-rw-r--r--   1 sznajder  staff  47753684  2 May  2022 jetImage_7_100p_30000_40000.h5
-rw-r--r--   1 sznajder  staff  48176508  2 May  2022 jetImage_7_100p_40000_50000.h5
-rw-r--r--   1 sznajder  staff  47905139  2 May  2022 jetImage_7_100p_50000_60000.h5
-rw-r--r--   1 sznajder  staff  47812902  2 May  2022 jetImage_7_100p_60000_70000.h5
-rw-r--r--   1 sznajder  staff  47649314  2 May  2022 jetImage_7_100p_70000_80000.h5
-rw-r--r--   1 sznajder  staff  47830990  2 May  2022 jetImage_7_100p_80000_90000.h5


0

In [3]:
target = np.array([])
jetImage = np.array([])
# we cannot load all data on Colab. So we just take a few files
datafiles = ['Data-MLtutorial/JetDataset/jetImage_7_100p_30000_40000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_60000_70000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_50000_60000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_10000_20000.h5',
             'Data-MLtutorial/JetDataset/jetImage_7_100p_0_10000.h5']
# if you are running locally, you can use the full dataset doing
# for fileIN in glob.glob("tutorials/HiggsSchool/data/*h5"):
for fileIN in datafiles:
    print("Appending %s" %fileIN)
    f = h5py.File(fileIN)
    myjetImage = np.array(f.get("jetImage"))
    mytarget = np.array(f.get('jets')[0:,-6:-1])
    jetImage = np.concatenate([jetImage, myjetImage], axis=0) if jetImage.size else myjetImage
    target = np.concatenate([target, mytarget], axis=0) if target.size else mytarget
    f.close()
print(target.shape, jetImage.shape)

Appending Data-MLtutorial/JetDataset/jetImage_7_100p_30000_40000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_60000_70000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_50000_60000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_10000_20000.h5
Appending Data-MLtutorial/JetDataset/jetImage_7_100p_0_10000.h5
(50000, 5) (50000, 100, 100)


The dataset consists of 50000 with up to 100 particles in each jet. These 100 particles have been used to fill the 100x100 jet images.

---

We now shuffle the data, splitting them into a training and a validation dataset with 2:1 ratio

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(jetImage, target, test_size=0.33)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
del jetImage, target

(33500, 100, 100) (16500, 100, 100) (33500, 5) (16500, 5)


In keras, images are representable as $n \times m \times k$ tensors, where $n \times m$ are the pixel dimenions and $k$ is the number of channels (e.g., 1 in a black\&while image, 3 for an RGB image). In our case, k=1. To comply to this, we add the channel index by reshaping the image dataset

In [5]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], X_val.shape[2], 1))
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(33500, 100, 100, 1) (16500, 100, 100, 1) (33500, 5) (16500, 5)


# Conv 2D model building

The main ingredients of a Conv2D layer are:

- **filter**: a *k x k’* matrix of weights (orange matrix in the picture below) that scans the image and performs a scalar product of each image block (this is also called *kernel*)
- **stride**: number of pixels the filter is shifted by (=1 in the image below)
- **padding**: the amount of pixels added to an image when it is being processed by the filter of a CNN (helps keeping information on the boundaries of the original image by allowing border pixels to be at the center of the filter)
    - *valid* means no padding (default setting)
    - *same* results in padding with zeros evenly to the left/right or up/down of the input image as needed to ensure that the output has the same shape as the input

<div>
<img src="https://raw.githubusercontent.com/jngadiub/ML_course_Pavia_23/main/neural-networks/conv2d.gif" width="400"/>
</div>

It is common practice to insert **pooling** layers in between Conv2D layers to progressively reduce the size of the representation and thus reduce the amount of parameters and computation in the network. Pooling also makes processing more robust to changes in the position of a feature in the image. Common types of pooling operations are:

- **MaxPooling**: given an image and a pool of size *k x k’*, scans the image and replaces each *k x k’* patch with its *maximum* -- helps to extract the sharpest features on the image when the sharpest features are a best lower-level representation of the image
- **AveragePooling**: given an image and a pool of size *k x k’*, scans the image and replaces each *k x k’* patch with its *average* -- helps to extract the smooth features when "colours" transition is smooth

<img src="https://github.com/jngadiub/ML_course_Pavia_23/blob/main/neural-networks/figures/pooling.png?raw=1" alt="Drawing" style="width: 800px;"/>

In [6]:
# keras imports
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Dense, Input, Conv2D, Dropout, Flatten
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN

In [7]:
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]
dropoutRate = 0.25

In [8]:
image_shape = (img_rows, img_cols, 1)
####
inputImage = Input(shape=(image_shape))
x = Conv2D(5, kernel_size=(5,5), data_format="channels_last", strides=(1, 1), padding="same")(inputImage)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D( pool_size = (5,5))(x)
x = Dropout(dropoutRate)(x)
#
x = Conv2D(3, kernel_size=(3,3), data_format="channels_last", strides=(1, 1), padding="same")(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D( pool_size = (3,3))(x)
x = Dropout(dropoutRate)(x)
#
x = Flatten()(x)
#
x = Dense(5, activation='relu')(x)
#
output = Dense(5, activation='softmax')(x)
####
model = Model(inputs=inputImage, outputs=output)

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 100, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 100, 100, 5)    │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 100, 5)    │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 100, 100, 5)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 20, 20, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 20, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 20, 20, 3)      │           138 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20, 20, 3)      │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 20, 20, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 108)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           545 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            30 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 875 (3.42 KB)

 Trainable params: 859 (3.36 KB)

 Non-trainable params: 16 (64.00 B)

We now train the model. This takes really long time and processing power on common CPUs. **If you are running locally set TRAIN=False** such that a pre-trained model is loaded for the next evaluation steps. We live as homework to reproduce the results (suggest to use Colab with GPU).

In [10]:
batch_size = 128
n_epochs = 10


history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose = 2,
                    validation_data=(X_val, y_val),
                    callbacks = [
                                 EarlyStopping(monitor='val_loss', patience=10, verbose=1),
                                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
                                TerminateOnNaN()]
                    )


Epoch 1/10
262/262 - 31s - 117ms/step - loss: 2.0267 - val_loss: 1.5229 - learning_rate: 0.0010
Epoch 2/10
262/262 - 29s - 111ms/step - loss: 1.4949 - val_loss: 1.4077 - learning_rate: 0.0010
Epoch 3/10
262/262 - 29s - 112ms/step - loss: 1.4152 - val_loss: 1.3401 - learning_rate: 0.0010
Epoch 4/10
262/262 - 30s - 113ms/step - loss: 1.3750 - val_loss: 1.3128 - learning_rate: 0.0010
Epoch 5/10
262/262 - 29s - 110ms/step - loss: 1.3462 - val_loss: 1.2871 - learning_rate: 0.0010
Epoch 6/10
262/262 - 29s - 112ms/step - loss: 1.3254 - val_loss: 1.2602 - learning_rate: 0.0010
Epoch 7/10
262/262 - 29s - 111ms/step - loss: 1.3086 - val_loss: 1.2478 - learning_rate: 0.0010
Epoch 8/10
262/262 - 29s - 111ms/step - loss: 1.2921 - val_loss: 1.2239 - learning_rate: 0.0010
Epoch 9/10
262/262 - 31s - 117ms/step - loss: 1.2717 - val_loss: 1.2160 - learning_rate: 0.0010
Epoch 10/10
262/262 - 30s - 114ms/step - loss: 1.2594 - val_loss: 1.2021 - learning_rate: 0.0010


In [11]:
# plot training history
history = pd.DataFrame(history.history)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.yscale('log')
plt.title('Training History')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

NameError: name 'TRAIN' is not defined

# Building the ROC Curves

In [ ]:
labels = ['gluon', 'quark', 'W', 'Z', 'top']

In [ ]:

from sklearn.metrics import roc_curve, auc
predict_val = model.predict(X_val)
df = pd.DataFrame()
fpr = {}
tpr = {}
auc1 = {}

plt.figure()
for i, label in enumerate(labels):
        df[label] = y_val[:,i]
        df[label + '_pred'] = predict_val[:,i]

        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])

        auc1[label] = auc(fpr[label], tpr[label])

        plt.plot(tpr[label],fpr[label],label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.))
plt.semilogy()
plt.xlabel("sig. efficiency")
plt.ylabel("bkg. mistag rate")
plt.ylim(0.000001,1)
plt.grid(True)
plt.legend(loc='lower right')
plt.show()